In [21]:
#加载数据
import pandas as pd
data = pd.read_csv('E:/Phone1.csv')

In [23]:
data.head()
data.shape

(41800, 22)

In [25]:
data.isnull().any(axis=0)  #年、月和年龄段三列存在缺失数据

订单号            False
订单日期           False
年               True
月               True
地区名字           False
省份名字           False
城市名字           False
品牌             False
型号             False
运行内存           False
机身内存           False
数量             False
用户名            False
用户姓名           False
年龄             False
年龄段             True
性别             False
手机号            False
价格             False
销售额            False
Unnamed: 20     True
Unnamed: 21     True
dtype: bool

In [69]:
#查看缺失数据的占比
for col in data.columns:
    if data[col].isnull().sum() >0: #条件满足则表示col列存在空值
        #计算col列的空值占比
        null_count = data[col].isnull().sum() #空值的数量
        null_rate = null_count / data[col].size
print(col,null_rate)

Unnamed: 21 1.0


In [71]:
data['订单日期'] = pd.to_datetime(data['订单日期'])
data['年'] = data['订单日期'].dt.year

In [73]:
#可以将订单日期中的年份和月份单独取出来，赋值到年和月两列中
data['年'] = data['订单日期'].dt.year
def get_month(d):
    d =str(d)
    year =d.split('-')[0]
    month =d.split('-')[1]
    return year + '-'+ month
data['月'] = data['订单日期'].map(get_month)

In [51]:
data['年龄'].describe()    #发现年龄是在16-49之间
#人为制定几个年龄段：16-25，26-35，36-49
data['年龄段']=pd.cut(data['年龄'],bins=[16,25,35,49])
#bins表示生成几个箱子：16-25，26-35，36-49

In [55]:
# 查看不同品牌手机的累计销量和累计销售额，并对累计销量进行降序排列
result = data.groupby(by='品牌')[['数量', '销售额']].sum()
result.rename(
    columns={'数量': '累计销量', '销售额': '累计销售额'},
    inplace=True
)
result.sort_values(by='累计销量', ascending=False, inplace=True)

# 这里可以根据需要添加代码来查看处理后的结果，比如打印前几行
print(result.head())

         累计销量     累计销售额
品牌                     
vivo    20601  60274031
小米      17889  41897903
iphone  14954  80227880
华为      14623  48727562
三星      13551  64473019


In [61]:
result = data.groupby(by='月')['数量'].sum().reset_index()
result.sort_values(by='数量', ascending=False, inplace=True)
print(result)

          月    数量
26  2020-03  5647
11  2018-12  5643
0   2018-01  5613
18  2019-07  5604
7   2018-08  5574
16  2019-05  5559
12  2019-01  5542
10  2018-11  5526
21  2019-10  5491
14  2019-03  5488
5   2018-06  5478
15  2019-04  5471
4   2018-05  5467
2   2018-03  5447
22  2019-11  5434
8   2018-09  5425
23  2019-12  5417
3   2018-04  5392
6   2018-07  5383
25  2020-02  5379
9   2018-10  5342
19  2019-08  5310
24  2020-01  5265
17  2019-06  5255
20  2019-09  5219
1   2018-02  5193
13  2019-02  4989


In [63]:
#不同年龄段的购买力
data.groupby(by='年龄段')['订单号'].count()

C:\Users\luoya\AppData\Local\Temp\ipykernel_33960\4197737856.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(by='年龄段')['订单号'].count()


年龄段
(16, 25]    21229
(25, 35]    19595
(35, 49]      462
Name: 订单号, dtype: int64

In [65]:
#查看不同城市的购买力情况
data.groupby(by='城市名字')['订单号'].count().sort_values(ascending=False)

城市名字
上海市     6326
北京市     6301
张家口市     259
秦皇岛市     249
石家庄市     248
        ... 
潍坊市       69
湛江市       69
兰州市       68
漳州市       66
许昌市       65
Name: 订单号, Length: 292, dtype: int64

In [67]:
#查看不同品牌的不同型号的最高和最低价格是多少
data.groupby(by=['品牌','型号'])['价格'].agg(['max','min'])

max    min
品牌     型号                             
iphone iPhone 11           5999   5999
       iPhone 11 Pro       9999   9999
       iPhone 11 Pro Max  10899  10899
       iPhone 6s Plus      1198   1198
       iPhone 7            2899   2899
...                         ...    ...
魅族     魅族16Xs              1499   1499
       魅族16sPro            2959   2959
       魅族16spro            2959   2959
       魅族16th              1988   1988
       魅族Note9 s           1026   1026

[114 rows x 2 columns]